**This Jupyter notebook should be run from within a compute instance on AzureML, in a Python kernel, specifically `Python 3.10 - SDK v2 (Python 3.10.11)`**. 

## Create a client connection to the AzureML workspace

The following cell creates a connection object called `ml_client` which has a connection to the AzureML workspace. You have to create this from every notebook or python script that interacts with the AzureML platform.

In [7]:
!python --version
!pip install azure-ai-ml --upgrade

Python 3.10.11
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 86.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.4/258.4 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 35.7 MB/s eta 0:00:00
  Attempting uninstall: azure-storage-blob
    Found existing installation: azure-storage-blob 12.13.0
    Uninstalling azure-storage-blob-12.13.0:
      Successfully uninstalled azure-storage-blob-12.13.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.57.0 requir

In [8]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import UserIdentityConfiguration

Use this authentication mechanism if you are running this notebook from your compute instance within Azure Machine Learning:

In [9]:
ml_client = MLClient.from_config(
    DefaultAzureCredential(),
)


Found the config file in: /config.json


However, you can also run this control plane notebook from your Laptop. You need to install the python libraries in the `requirements.txt` file.

In [10]:
ml_client = MLClient(
    credential=DefaultAzureCredential(),
    workspace_name="project-group-35",
    subscription_id="21ff0fc0-dd2c-450d-93b7-96eeb3699b22",
    resource_group_name="project-group-35"
)

## Download the Spark-NLP jar to your working directory to be able to to add to the job cluster.

You only need to do this once. The jar file needs to be in the same directory of the script that will be run as a job.

In [11]:
# Download the spark-nlp jar and save it locally. This needs to be done before submitting a job.
import requests
response = requests.get("https://repo1.maven.org/maven2/com/johnsnowlabs/nlp/spark-nlp_2.12/5.0.2/spark-nlp_2.12-5.0.2.jar")
with open("spark-nlp_2.12-5.0.2.jar", "wb") as f:
    f.write(response.content)

## Define the Job

The following cell defines the job. It is an object of [Spark Class](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.spark?view=azure-python) that contains the required information to run a job:

For more information about the parameters used in the job definition, [read the documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-submit-spark-jobs?view=azureml-api-2&tabs=sdk#submit-a-standalone-spark-job).

In [13]:
sparknlp_job_def = spark(
    display_name="last-test-expect-success",  
    code="./",
    entry={"file": "Users/tvl8/fall-2024-project-team-35/code/spark-job-with-sparknlp-copy/sample-spark-nlp-job.py"},
    driver_cores=1,
    driver_memory="7g",
    executor_instances=1,
    executor_cores=1,
    executor_memory="7g",
        resources={
        "instance_type": "Standard_E4S_V3",
        "runtime_version": "3.4",
    },
    jars=["spark-nlp_2.12-5.0.2.jar"],
    environment="sparknlp-python-env@latest",
    identity=UserIdentityConfiguration()
)

## Submit the job

The following cell takes the job you defined above and submits it. If you are submitting multiple jobs, you may want to create separate job definition objects for clarity. You can submit more than one job, just remember that each job will spin up a Spark cluster.

In [16]:
import os
print("File exists:", os.path.exists("sample-spark-nlp-job.py"))

File exists: True


In [17]:
sparknlp_job = ml_client.jobs.create_or_update(sparknlp_job_def)

MlException: {
  "result": "Failed",
  "errors": [
    {
      "message": "Entry /mnt/batch/tasks/shared/LS_root/mounts/clusters/tvl8-ci01/code/Users/tvl8/fall-2024-project-team-35/code/spark-job-with-sparknlp-copy/Users/tvl8/fall-2024-project-team-35/code/spark-job-with-sparknlp-copy/sample-spark-nlp-job.py doesn't exist.",
      "path": "component.entry",
      "value": {
        "file": "Users/tvl8/fall-2024-project-team-35/code/spark-job-with-sparknlp-copy/sample-spark-nlp-job.py"
      }
    }
  ]
}

## Get the Job Studio URL

Once you submit the job, you can navigate to it in the AzureML Studio and monitor it's progress. There are ways to do it through the SDK but for now just use the Studio. These are unattended jobs, which means you can shut down this notebook and the Compute Instance, but the job will go through it's lifecycle:

- Job is submitted
- Job is queued
- Job is run
- Job completes (assuming no errors)

**Each job's Studio URL will be different.**

In [ ]:
print(sparknlp_job.studio_url)